In [1]:
# Load Artifacts
import json, joblib, numpy as np
from pathlib import Path

OUT_DIR = Path(r"C:\Users\Nicee\Desktop\kenkyu\gnamboost_outputs")
INT_DIR = OUT_DIR / "interim"
FIG_DIR = OUT_DIR / "figs"
FIG_DIR.mkdir(parents=True, exist_ok=True)

Xtr = joblib.load(INT_DIR/"Xtr.pkl"); Xva = joblib.load(INT_DIR/"Xva.pkl"); Xte = joblib.load(INT_DIR/"Xte.pkl")
ytr = joblib.load(INT_DIR/"ytr.npy"); yva = joblib.load(INT_DIR/"yva.npy"); yte = joblib.load(INT_DIR/"yte.npy")
Xtr_t = joblib.load(INT_DIR/"Xtr_t.npy"); Xva_t = joblib.load(INT_DIR/"Xva_t.npy"); Xte_t = joblib.load(INT_DIR/"Xte_t.npy")
preproc = joblib.load(INT_DIR/"preproc.joblib")
meta = json.load(open(INT_DIR/"meta.json","r",encoding="utf-8"))

feature_cols = meta["feature_cols"]; num_cols = meta["num_cols"]; bin_cols = meta["bin_cols"]; cat_cols = meta["cat_cols"]


In [2]:
# Cell 1 Logistic & XGBoost Baselines
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import xgboost as xgb

# Logistic
logit = LogisticRegression(penalty="l2", C=0.001, solver="lbfgs", max_iter=2000)
logit.fit(Xtr_t, ytr)
p_va_logit = logit.predict_proba(Xva_t)[:,1]
p_te_logit_raw = logit.predict_proba(Xte_t)[:,1]
iso_logit = IsotonicRegression(out_of_bounds="clip").fit(p_va_logit, yva)
p_logit = iso_logit.transform(p_te_logit_raw)
print("Logistic: AUROC=", roc_auc_score(yte,p_logit), " AP=", average_precision_score(yte,p_logit))

# XGBoost + early stop
xgb_model = xgb.XGBClassifier(
    grow_policy="lossguide", max_depth=0, max_leaves=64,
    learning_rate=0.01, n_estimators=20000, subsample=0.75,
    colsample_bytree=0.75, colsample_bylevel=0.85,
    min_child_weight=5, reg_alpha=0.15, reg_lambda=2.5,
    gamma=0.0, max_bin=256, max_delta_step=1,
    tree_method="hist", n_jobs=-1, random_state=meta["seed"],
    early_stopping_rounds=1200,
    eval_metric="auc"
)
xgb_model.fit(Xtr_t, ytr, eval_set=[(Xva_t,yva)], verbose=False)
best_iter = xgb_model.best_iteration
booster = xgb_model.get_booster()
dva = xgb.DMatrix(Xva_t); dte = xgb.DMatrix(Xte_t)
p_va_xgb = booster.predict(dva, iteration_range=(0, best_iter+1))
p_te_xgb_raw = booster.predict(dte, iteration_range=(0, best_iter+1))
iso_xgb = IsotonicRegression(out_of_bounds="clip").fit(p_va_xgb, yva)
p_xgb = iso_xgb.transform(p_te_xgb_raw)
print("XGB: AUROC=", roc_auc_score(yte,p_xgb), " AP=", average_precision_score(yte,p_xgb))

# Save predictions & models
joblib.dump(p_logit, INT_DIR/"p_logit.npy")
joblib.dump(p_xgb,   INT_DIR/"p_xgb.npy")
joblib.dump(logit,   INT_DIR/"logit_model.joblib")
booster.save_model(str(INT_DIR/"xgb_booster.json"))
joblib.dump({"best_iteration": best_iter}, INT_DIR/"xgb_info.joblib")


Logistic: AUROC= 0.7795568561576975  AP= 0.6232944200494668
XGB: AUROC= 0.7984220843281925  AP= 0.661829642863953


['C:\\Users\\Nicee\\Desktop\\kenkyu\\gnamboost_outputs\\interim\\xgb_info.joblib']

In [3]:
# Cell 2 GNAM training
import os, math, numpy as np, contextlib, torch, joblib
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import scipy.special as sps
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.isotonic import IsotonicRegression

os.environ["OMP_NUM_THREADS"] = "12"
os.environ["MKL_NUM_THREADS"] = "12"
try:
    torch.set_num_threads(12); torch.set_num_interop_threads(4)
except Exception:
    pass

def get_amp_ctx():
    try:
        return torch.autocast(device_type="cpu", dtype=torch.bfloat16)
    except Exception:
        return contextlib.nullcontext()
amp_ctx = get_amp_ctx()

class ExULayer(nn.Module):
    def __init__(self, nonlin="softplus"):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(1))
        self.b = nn.Parameter(torch.zeros(1))
        self.nonlin = nonlin
    def forward(self, x):
        z = torch.exp(self.w) * (x - self.b)
        if self.nonlin == "softplus": return F.softplus(z)
        if self.nonlin == "tanh": return torch.tanh(z)
        return F.relu(z)

class FeatureNet(nn.Module):
    def __init__(self, hidden=96, dropout=0.20):
        super().__init__()
        self.exu = ExULayer("softplus")
        self.fc1 = nn.Linear(1, hidden)
        self.fc2 = nn.Linear(hidden, hidden // 2)
        self.fc3 = nn.Linear(hidden // 2, 1)
        self.dp = nn.Dropout(dropout)
    def forward(self, x):
        h = self.exu(x)
        h = F.relu(self.fc1(h))
        h = self.dp(F.relu(self.fc2(h)))
        return self.fc3(h)

class GNAM(nn.Module):
    def __init__(self, n_features, hidden=96, dropout=0.20):
        super().__init__()
        self.bias = nn.Parameter(torch.zeros(1))
        self.fnets = nn.ModuleList([FeatureNet(hidden, dropout) for _ in range(n_features)])
    def forward(self, x):
        outs = [self.fnets[j](x[:, j:j+1]) for j in range(x.shape[1])]
        eta = self.bias + torch.stack(outs, dim=2).sum(dim=2)
        return eta.squeeze(1)

def to_loader(X, y, batch=1024, shuffle=True):
    ds = TensorDataset(torch.tensor(X, dtype=torch.float32),
                       torch.tensor(y, dtype=torch.float32))
    return DataLoader(ds, batch_size=batch, shuffle=shuffle, drop_last=False, num_workers=0)

def train_gnam(model, tr_loader, va_loader, max_epochs=600, lr=5e-4, weight_decay=1e-4, patience=200,
               device="cpu", pos_weight=None, logit_clamp=10.0, grad_clip_norm=5.0, report_every=20):
    import time
    import numpy as np
    import torch
    import torch.nn.functional as F
    from sklearn.metrics import roc_auc_score, average_precision_score

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = (nn.BCEWithLogitsLoss(pos_weight=torch.tensor(float(pos_weight), device=device))
                 if pos_weight is not None else nn.BCEWithLogitsLoss())
    best_loss, bad_epochs, best_state = np.inf, 0, None
    model.to(device)

    for ep in range(max_epochs):
        t0 = time.time()

        # Train
        model.train()
        tr_loss_sum, tr_n = 0.0, 0
        for xb, yb in tr_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with amp_ctx:
                logits = model(xb)
                if logit_clamp is not None:
                    logits = torch.clamp(logits, -abs(logit_clamp), abs(logit_clamp))
                loss = criterion(logits, yb)
            loss.backward()
            if grad_clip_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
            opt.step()
            tr_loss_sum += float(loss.item()) * len(xb)
            tr_n += len(xb)
        tr_loss = tr_loss_sum / max(tr_n, 1)

        # Validation
        model.eval()
        va_loss_sum, va_n = 0.0, 0
        logits_all, y_all = [], []
        with torch.no_grad():
            for xb, yb in va_loader:
                xb, yb = xb.to(device), yb.to(device)
                with amp_ctx:
                    logits = model(xb)
                    if logit_clamp is not None:
                        logits = torch.clamp(logits, -abs(logit_clamp), abs(logit_clamp))
                    vloss = F.binary_cross_entropy_with_logits(logits, yb).item()
                va_loss_sum += vloss * len(xb)
                va_n += len(xb)
                logits_all.append(logits.detach().cpu().numpy().ravel())
                y_all.append(yb.detach().cpu().numpy().ravel())
        avg_val_loss = va_loss_sum / max(va_n, 1)

        # Metrics on validation
        y_cat = np.concatenate(y_all) if y_all else np.array([])
        p_cat = 1.0 / (1.0 + np.exp(-np.concatenate(logits_all))) if logits_all else np.array([])
        auroc = roc_auc_score(y_cat, p_cat) if len(np.unique(y_cat)) > 1 else np.nan
        ap = average_precision_score(y_cat, p_cat) if len(np.unique(y_cat)) > 1 else np.nan

        # Report every N epochs
        if ((ep + 1) % report_every) == 0:
            print(f"Epoch {ep+1:03d}/{max_epochs} | "
                  f"train_loss={tr_loss:.5f} | val_loss={avg_val_loss:.5f} | "
                  f"AUROC={auroc:.4f} | AP={ap:.4f} | "
                  f"bad_epochs={bad_epochs} | time={time.time()-t0:.1f}s")

        # Early stop
        if avg_val_loss < best_loss - 1e-6:
            best_loss, bad_epochs = avg_val_loss, 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print(f"Early stop @ epoch {ep+1}, best_val_loss={best_loss:.5f}")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

device = "cpu"
pos_rate = float(np.mean(ytr))
pos_weight = (1.0 - pos_rate) / max(pos_rate, 1e-6)

gnam = GNAM(n_features=Xtr_t.shape[1], hidden=96, dropout=0.20)
tr_loader = to_loader(Xtr_t, ytr, batch=1024, shuffle=True)
va_loader = to_loader(Xva_t, yva, batch=2048, shuffle=False)

gnam = train_gnam(
    gnam, tr_loader, va_loader,
    max_epochs=600, lr=5e-4, weight_decay=1e-4, patience=200,
    device=device, pos_weight=pos_weight, logit_clamp=10.0, grad_clip_norm=5.0,
    report_every=20
)

torch.save(gnam.state_dict(), INT_DIR/"gnam_best.pt")

gnam.eval()
with torch.no_grad(), amp_ctx:
    eta_tr = gnam(torch.tensor(Xtr_t, dtype=torch.float32, device=device)).cpu().numpy().astype(np.float32)
    eta_va = gnam(torch.tensor(Xva_t, dtype=torch.float32, device=device)).cpu().numpy().astype(np.float32)
    eta_te = gnam(torch.tensor(Xte_t, dtype=torch.float32, device=device)).cpu().numpy().astype(np.float32)

p_tr_gnam = sps.expit(eta_tr)
p_va_gnam = sps.expit(eta_va)
p_te_gnam = sps.expit(eta_te)
iso_gnam = IsotonicRegression(out_of_bounds="clip").fit(p_va_gnam, yva)
p_gnam = iso_gnam.transform(p_te_gnam)

print("GNAM raw (valid AUROC/AP):", roc_auc_score(yva, p_va_gnam), average_precision_score(yva, p_va_gnam))
print("GNAM test (raw AUROC/AP):", roc_auc_score(yte, p_te_gnam), average_precision_score(yte, p_te_gnam))

joblib.dump({"eta_tr": eta_tr, "eta_va": eta_va, "eta_te": eta_te}, INT_DIR/"gnam_eta.joblib")
joblib.dump(p_gnam, INT_DIR/"p_gnam.npy")


Epoch 020/600 | train_loss=0.73667 | val_loss=0.57059 | AUROC=0.7843 | AP=0.6322 | bad_epochs=16 | time=25.6s
Epoch 040/600 | train_loss=0.73645 | val_loss=0.55465 | AUROC=0.7842 | AP=0.6329 | bad_epochs=36 | time=23.7s
Epoch 060/600 | train_loss=0.73548 | val_loss=0.55984 | AUROC=0.7843 | AP=0.6325 | bad_epochs=56 | time=23.9s
Epoch 080/600 | train_loss=0.73498 | val_loss=0.56118 | AUROC=0.7849 | AP=0.6340 | bad_epochs=76 | time=24.0s
Epoch 100/600 | train_loss=0.73530 | val_loss=0.57030 | AUROC=0.7845 | AP=0.6335 | bad_epochs=96 | time=24.1s
Epoch 120/600 | train_loss=0.73493 | val_loss=0.55416 | AUROC=0.7848 | AP=0.6339 | bad_epochs=116 | time=24.2s
Epoch 140/600 | train_loss=0.73500 | val_loss=0.55810 | AUROC=0.7847 | AP=0.6339 | bad_epochs=136 | time=24.4s
Epoch 160/600 | train_loss=0.73496 | val_loss=0.56936 | AUROC=0.7849 | AP=0.6347 | bad_epochs=156 | time=24.1s
Epoch 180/600 | train_loss=0.73514 | val_loss=0.57015 | AUROC=0.7843 | AP=0.6330 | bad_epochs=176 | time=24.4s
Epoch 

['C:\\Users\\Nicee\\Desktop\\kenkyu\\gnamboost_outputs\\interim\\p_gnam.npy']

In [4]:
# Cell 3 GNAM–Boost (XGB on GNAM base_margin)
import xgboost as xgb
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, log_loss
import numpy as np, joblib

etas = joblib.load(INT_DIR/"gnam_eta.joblib")
eta_tr, eta_va, eta_te = etas["eta_tr"], etas["eta_va"], etas["eta_te"]

dtr2 = xgb.DMatrix(Xtr_t, label=ytr); dtr2.set_base_margin(eta_tr.astype(np.float32))
dva2 = xgb.DMatrix(Xva_t, label=yva); dva2.set_base_margin(eta_va.astype(np.float32))
dte2 = xgb.DMatrix(Xte_t, label=yte); dte2.set_base_margin(eta_te.astype(np.float32))

pos_rate=float(ytr.mean())
scale_pos_weight=((1.0-pos_rate)/max(pos_rate,1e-9))**0.5

params2 = {
    "objective":"binary:logistic",
    "eval_metric":["aucpr","auc","logloss"],
    "grow_policy":"lossguide","max_depth":0,"max_leaves":36,"eta":0.05,
    "subsample":0.7,"colsample_bytree":0.6,"colsample_bylevel":0.8,
    "min_child_weight":15,"reg_alpha":0.20,"reg_lambda":3.0,"max_delta_step":1,
    "scale_pos_weight":scale_pos_weight,"tree_method":"hist"
}
bst2 = xgb.train(params2, dtr2, num_boost_round=6000, evals=[(dtr2,"train"),(dva2,"valid")],
                 early_stopping_rounds=800, verbose_eval=200)

best_iter = bst2.best_iteration if hasattr(bst2,"best_iteration") else None
p_va_gb = bst2.predict(dva2, iteration_range=(0, best_iter+1) if best_iter is not None else None)
p_te_gb = bst2.predict(dte2, iteration_range=(0, best_iter+1) if best_iter is not None else None)
iso_gb = IsotonicRegression(out_of_bounds="clip").fit(p_va_gb, yva)
p_final = iso_gb.transform(p_te_gb)

print("GNAM–Boost:",
      "AUROC =", roc_auc_score(yte,p_final),
      "AP =", average_precision_score(yte,p_final),
      "Brier =", brier_score_loss(yte,p_final),
      "LogLoss =", log_loss(yte,p_final))

# Save
bst2.save_model(str(INT_DIR/"gnamboost_bst.json"))
joblib.dump(p_final, INT_DIR/"p_final.npy")


[0]	train-aucpr:0.63466	train-auc:0.78195	train-logloss:0.53570	valid-aucpr:0.63093	valid-auc:0.78350	valid-logloss:0.53377
[200]	train-aucpr:0.67505	train-auc:0.80163	train-logloss:0.51542	valid-aucpr:0.65816	valid-auc:0.79563	valid-logloss:0.52095
[400]	train-aucpr:0.69111	train-auc:0.81047	train-logloss:0.50612	valid-aucpr:0.66247	valid-auc:0.79761	valid-logloss:0.51853
[600]	train-aucpr:0.70311	train-auc:0.81744	train-logloss:0.49876	valid-aucpr:0.66386	valid-auc:0.79831	valid-logloss:0.51758
[800]	train-aucpr:0.71309	train-auc:0.82340	train-logloss:0.49241	valid-aucpr:0.66435	valid-auc:0.79869	valid-logloss:0.51693
[1000]	train-aucpr:0.72190	train-auc:0.82858	train-logloss:0.48679	valid-aucpr:0.66440	valid-auc:0.79858	valid-logloss:0.51688
[1200]	train-aucpr:0.72953	train-auc:0.83334	train-logloss:0.48162	valid-aucpr:0.66437	valid-auc:0.79846	valid-logloss:0.51687
[1400]	train-aucpr:0.73709	train-auc:0.83771	train-logloss:0.47676	valid-aucpr:0.66415	valid-auc:0.79830	valid-logloss

['C:\\Users\\Nicee\\Desktop\\kenkyu\\gnamboost_outputs\\interim\\p_final.npy']